<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# [Ateliers: Technologies des grosses data](https://github.com/wikistat/Ateliers-Big-Data)

# [Reconnaissance d'Activité Humaine](https://github.com/wikistat/Ateliers-Big-Data/5-HumanActivityRecognition) ([*HAR*](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones)) en <a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 120px; display: inline" alt="Python"/></a>  
##  Seconde partie:  apprentissage (profond) des signaux bruts  avec <a href="https://keras.io/"><img src="https://s3.amazonaws.com/keras.io/img/keras-logo-2018-large-1200.png" style="max-width: 100px; display: inline" alt="Keras"/></a>

Ce notebook présente la partie prediction de l'activité. Pour l'exploration, se référer au calepin afférent.

##  1 Introduction
###  1.1 Contexte
Les données sont issues de la communauté qui vise la reconnaissance d'activités humaines (*Human activity recognition, HAR*) à partir d’enregistrements, par exemple du gyroscope et de l'accéléromètre d'un smartphone.
Voir à ce propos l'[article](https://www.elen.ucl.ac.be/Proceedings/esann/esannpdf/es2013-11.pdf) relatant un colloque de 2013.  

Les données publiques disponibles ont été acquises, décrites et analysées par [Anguita et al. (2013)](https://www.elen.ucl.ac.be/Proceedings/esann/esannpdf/es2013-84.pdf). Elles sont accessibles sur le [dépôt](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones) de l'University California Irvine (UCI) consacré à l'apprentissage machine ainsi que sur le site *Kaggle*.

L'archive contient les données brutes: accélérations en x, y, et z, chacun de 128 colonnes. D'autres fichiers en y soustrayant la gravité naturelle ainsi que les accélérations angulaires en x, y, et z, soit en tout 9 fichiers. Mais 6 utiles avec 6*128=768 mesures.

Les méthodes d'apprentissage sont appliquées sur ces données brutes, sans calculs préliminaires de caractéristiques (*features*).

### 1.2 Objectif
Cette deuxième étape s'intéresse aux données brutes. Est-il possible d'économiser le travail préliminaire de définition des variables métier en utilisant, par exemple, les ressources de décompositions systématiques sur une base d'ondelette ou un algorihtme d'apprentissage profond?

**Objecctif** Faire aussi bien (96% de bien classés) qu'avec les variables métier.

### 1.3 Travail à réaliser
**Attention l'accès à un environnement *GPU* est très vivement conseillé voire indispensable.**
- Modélisation, prévision de l'échantillon test par
   - Régression logistique (`Scikit-learn`)
   - Apprentissage profond en utilisant `Keras` 
       - MLP sur signaux "applatis"
       - MLP sur signaux mutlidimensionelles
       - LSTM
       - 1D Convolution
       - 2D Convolution
   
- Ajouter à ce calepin: 
    - Application des méthodes d'apprentissage classique ou non sur les coefficients des décompositions des signaux en ondelettes
    - optimisation des paramètres des différentes méthodes.
    - Améliorer l'architexture des réseaux?


## 2 Mise en place
### 2.1 Librairies et initialisation

In [22]:
import pandas as pd
import numpy as np
import os
import time
import copy
import random
import itertools

#Utils Sklearn
import sklearn.linear_model as lm
from sklearn.metrics import confusion_matrix

%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sb
sb.set()

# DEEP LEARING
import tensorflow as tf
np.random.seed(42)
tf.set_random_seed(42)

# for reproducibility
# https://github.com/fchollet/keras/issues/2280
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

import keras.models as km 
import keras.layers as kl 
import keras.layers.core as klc

### 2.2 Prise en charge des données
#### Sources

Les données sont celles originales du dépôt de l'[UCI](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones). Elle peuvent être téléchargées en cliquant [ici](https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip).

Elles contiennent deux jeux de dimensions différentes, chacun partagé en apprentissage et test.

1. Multidimensionel: un individus est constitué de 9 Séries Temporelles de *dimensions* $(N, 128, 9)$
2. Unidimensionnel: Les 9 Séries Temporelles sont concaténées pour constituer un vecteur de 128*9 = 1152 variables de *dimensions* $(N, 1152)*
        
Deux objets différents sont construits pour définir la variable $Y$ réponse car les librairies `Scikit-learn` et `Keras` prennent en compte des structures différentes: 
    
1. `Scikit-Learn`  Un vecteur de dimension $(N, 1)$ avec, pour chaque individu le numéro du label de l'activité de 0 à 5.
2. `Keras` Une matrice de dimension $(N, 6)$ des indicatrices (0 ou 1) des modalités de $Y$.

#### Lecture des données

In [ ]:
DATADIR_UCI = '~/DeepLearning/data/HARWS//UCI HAR Dataset'

SIGNALS = [ "body_acc_x", "body_acc_y", "body_acc_z", "body_gyro_x", "body_gyro_y", "body_gyro_z", "total_acc_x", "total_acc_y", "total_acc_z"]

def my_read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

def load_signal(data_dir, subset, signal):
    filename = f'{data_dir}/{subset}/Inertial Signals/{signal}_{subset}.txt'
    x = my_read_csv(filename).as_matrix()
    return x 

def load_signals(data_dir, subset, flatten = False):
    signals_data = []
    for signal in SIGNALS:
        signals_data.append(load_signal(data_dir, subset, signal)) 
    
    if flatten :
        X = np.hstack(signals_data)
    else:
        X = np.transpose(signals_data, (1, 2, 0))
        
    return X 

def load_y(data_dir, subset, dummies = False):
    filename = f'{data_dir}/{subset}/y_{subset}.txt'
    y = my_read_csv(filename)[0]
    
    
    if dummies:
        Y = pd.get_dummies(y).as_matrix()
    else:
        Y = y.as_matrix()
    
    return Y

Vérification des dimensions

In [23]:
#Multidimensional Data
X_train, X_test = load_signals(DATADIR_UCI, 'train'), load_signals(DATADIR_UCI, 'test')
# Flattened Data
X_train_flatten, X_test_flatten = load_signals(DATADIR_UCI, 'train', flatten=True), load_signals(DATADIR_UCI, 'test', flatten=True)

# Label Y
Y_train_label, Y_test_label = load_y(DATADIR_UCI, 'train', dummies = False), load_y(DATADIR_UCI, 'test', dummies = False)
#Dummies Y (For Keras)
Y_train_dummies, Y_test_dummies = load_y(DATADIR_UCI, 'train', dummies = True), load_y(DATADIR_UCI, 'test', dummies = True)

N_train = X_train.shape[0]
N_test = X_test.shape[0]

In [24]:
print("Dimension")
print("Données Multidimensionelles, : " + str(X_train.shape))
print("Données Unimensionelles, : " + str(X_train_flatten.shape))
print("Vecteur réponse (scikit-learn) : " + str(Y_train_label.shape))
print("Matrice réponse(Keras) : " + str(Y_train_dummies.shape))

Dimension
Données Multidimensionelles, : (7352, 128, 9)
Données Unimensionelles, : (7352, 1152)
Vecteur réponse (scikit-learn) : (7352,)
Matrice réponse(Keras) : (7352, 6)


#### Utilitaires

In [25]:
DATA_DIR = "/Users/bguillouet/Insa/DeepLearning/data/HARWS/"
LABELS = ["WALKING","WALKING UPSTAIRS","WALKING DOWNSTAIRS","SITTING","STANDING","LAYING"]
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}


def my_confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

def _count_classes(y):
    return len(set([tuple(category) for category in y]))

## 3 Apprentissage des signaux uni-dimensionels

La base d'apprentissage est de dimension (`N_train`, 1152)

### 3.1 Régression Logistique

La Régression Logistique est une des méthodes conduisant aux meilleurs résultats sur les vairables métier.

In [21]:
t_start = time.time()
model_lr = lm.LogisticRegression(verbose=1)
model_lr.fit(X_train_flatten, Y_train_label)
t_end = time.time()
t_learning = t_end-t_start
score = model_lr.score(X_test_flatten, Y_test_label)
print("Score With Logistic Regression on Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
lr_prediction_label = model_lr.predict(X_test_flatten)
metadata_svc = {"time_learning" : t_learning, "score" : score}
pd.DataFrame(confusion_matrix(lr_prediction_label, Y_test_label), index = LABELS, columns=LABELS)

[LibLinear]Score With Logistic Regression on Inertial Signals = 57.45, Learning time = 33.21 secondes


,WALKING,WALKING UPSTAIRS,WALKING DOWNSTAIRS,SITTING,STANDING,LAYING
WALKING,120,63,97,0,1,0
WALKING UPSTAIRS,74,218,56,23,72,27
WALKING DOWNSTAIRS,93,66,103,1,2,0
SITTING,80,32,58,397,112,0
STANDING,129,92,106,70,345,0
LAYING,0,0,0,0,0,510


**Q** Que dire de la performance?

**TODO?** Tester d'autre méthode (SVM, XGBOOST?)

### 3.2 Perceptron multicouche

Un réseau de neurones classique est appris sur les données au même format que précédemment.

**Q** Expliciter les choix des paramètres et donc la structure du réseau.

In [27]:
epochs = 10
batch_size = 32
n_hidden = 32

n_features = X_train_flatten.shape[1]
n_classes=6


model_base_mlp_u =km.Sequential()
model_base_mlp_u.add(kl.Dense(n_hidden, input_shape=(n_features,),  activation = "relu"))
model_base_mlp_u.add(kl.Dropout(0.5))
model_base_mlp_u.add(kl.Dense(n_classes, activation='softmax'))
model_base_mlp_u.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model_base_mlp_u.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 32)                36896     
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 6)                 198       
Total params: 37,094
Trainable params: 37,094
Non-trainable params: 0
_________________________________________________________________


In [28]:
t_start = time.time()
model_base_mlp_u.fit(X_train_flatten,  Y_train_dummies, batch_size=batch_size, validation_data=(X_test_flatten, Y_test_dummies), epochs=epochs)
t_end = time.time()
t_learning = t_end-t_start

score = model_base_mlp_u.evaluate(X_test_flatten, Y_test_dummies)[1] 
print("Score With Simple MLP on Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
metadata_mlp_u = {"time_learning" : t_learning, "score" : score}
base_mlp_u_prediction = model_base_mlp_u.predict(X_test_flatten)

my_confusion_matrix(Y_test_dummies, base_mlp_u_prediction)

Train on 7352 samples, validate on 2947 samples
Epoch 1/10
7352/7352 [==============================] - 2s - loss: 1.0896 - acc: 0.5718 - val_loss: 0.8785 - val_acc: 0.7530
Epoch 2/10
7352/7352 [==============================] - 0s - loss: 0.8247 - acc: 0.6882 - val_loss: 0.7398 - val_acc: 0.7876
Epoch 3/10
7352/7352 [==============================] - 0s - loss: 0.7230 - acc: 0.7339 - val_loss: 0.6610 - val_acc: 0.8124
Epoch 4/10
7352/7352 [==============================] - 0s - loss: 0.6697 - acc: 0.7538 - val_loss: 0.6543 - val_acc: 0.8198
Epoch 5/10
7352/7352 [==============================] - 0s - loss: 0.6294 - acc: 0.7674 - val_loss: 0.6185 - val_acc: 0.8239
Epoch 6/10
7352/7352 [==============================] - 0s - loss: 0.5986 - acc: 0.7792 - val_loss: 0.5699 - val_acc: 0.8378
Epoch 7/10
7352/7352 [==============================] - 0s - loss: 0.5769 - acc: 0.7855 - val_loss: 0.6064 - val_acc: 0.8256
Epoch 8/10
7352/7352 [==============================] - 0s - loss: 0.5615 - a

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,520,0,0,0,0,17
SITTING,0,346,140,2,0,3
STANDING,0,83,439,0,3,7
WALKING,0,12,1,418,54,11
WALKING_DOWNSTAIRS,0,3,1,33,361,22
WALKING_UPSTAIRS,0,3,0,47,61,360


** Q ** : Que conclure sur ces résultats en terme de performance, de temps d'apprentissage? Comparer avec la regression logistique?

** Exo ** : Quel est l'influence de l'ajout de nouvelle couche? Supression du Dropout? ...

### 3.3 Réseau avec couche convolutionelle (*ConvNet*)

In [18]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 6

#else:
model_base_conv_1D =km.Sequential()
model_base_conv_1D.add(kl.Conv1D(32, 9, activation='relu', input_shape=(timesteps, input_dim)))
model_base_conv_1D.add(kl.MaxPooling1D(pool_size=3))
model_base_conv_1D.add(kl.Flatten())
model_base_conv_1D.add(kl.Dense(n_classes, activation='softmax'))
model_base_conv_1D.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

t_start = time.time()
model_base_conv_1D.fit(X_train,  Y_train_dummies, batch_size=batch_size, validation_data=(X_test, Y_test_dummies), epochs=epochs)
t_end = time.time()
t_learning = t_end-t_start

score = model_base_conv_1D.evaluate(X_test, Y_test_dummies)[1] 
print("Score With Conv on Multidimensional Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
metadata_conv = {"time_learning" : t_learning, "score" : score}
base_conv_1D_prediction = model_base_conv_1D.predict(X_test)

my_confusion_matrix(Y_test_dummies, base_conv_1D_prediction)

Train on 7352 samples, validate on 2947 samples
Epoch 1/10
7352/7352 [==============================] - 2s - loss: 0.5387 - acc: 0.8001 - val_loss: 0.6148 - val_acc: 0.7306
Epoch 2/10
7352/7352 [==============================] - 1s - loss: 0.2015 - acc: 0.9289 - val_loss: 0.3446 - val_acc: 0.8795
Epoch 3/10
7352/7352 [==============================] - 1s - loss: 0.1473 - acc: 0.9446 - val_loss: 0.2988 - val_acc: 0.8951
Epoch 4/10
7352/7352 [==============================] - 1s - loss: 0.1356 - acc: 0.9445 - val_loss: 0.2744 - val_acc: 0.9023
Epoch 5/10
7352/7352 [==============================] - 1s - loss: 0.1263 - acc: 0.9453 - val_loss: 0.3197 - val_acc: 0.9050
Epoch 6/10
7352/7352 [==============================] - 1s - loss: 0.1245 - acc: 0.9502 - val_loss: 0.3189 - val_acc: 0.9162
Epoch 7/10
7352/7352 [==============================] - 1s - loss: 0.1261 - acc: 0.9455 - val_loss: 0.3360 - val_acc: 0.9084
Epoch 8/10
7352/7352 [==============================] - 1s - loss: 0.1334 - a

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,510,0,0,0,0,27
SITTING,0,341,142,0,0,8
STANDING,0,37,493,1,0,1
WALKING,0,0,0,486,7,3
WALKING_DOWNSTAIRS,0,0,0,2,416,2
WALKING_UPSTAIRS,0,0,0,5,17,449


In [ ]:
model_base_conv_1D.summary()

## 4 Apprentissage des signaux multidimensionnels
Les différents signaux ne sont pas concaténées en un seul signal mais pris en compte parallèlement.

### 4.1 Perceptron multichouche
**Q** Expliciter les choix des paramètres et donc la structure du réseau.

In [12]:
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 6

model_base_mlp =km.Sequential()
model_base_mlp.add(kl.Dense(n_hidden, input_shape=(timesteps, input_dim),  activation = "relu"))
model_base_mlp.add(kl.Reshape((timesteps*n_hidden,) , input_shape= (timesteps, n_hidden)  ))
model_base_mlp.add(kl.Dense(n_classes, activation='softmax'))

model_base_mlp.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

t_start = time.time()
model_base_mlp.fit(X_train,  Y_train_dummies, batch_size=batch_size, validation_data=(X_test, Y_test_dummies), epochs=epochs)
t_end = time.time()
t_learning = t_end-t_start

score = model_base_mlp.evaluate(X_test, Y_test_dummies)[1] 
print("Score With Simple MLP on Multidimensional Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
metadata_mlp = {"time_learning" : t_learning, "score" : score}
base_mlp_prediction = model_base_mlp.predict(X_test)

my_confusion_matrix(Y_test_dummies, base_mlp_prediction)

Train on 7352 samples, validate on 2947 samples
Epoch 1/10
7352/7352 [==============================] - 0s - loss: 0.6266 - acc: 0.7414 - val_loss: 0.7181 - val_acc: 0.7302
Epoch 2/10
7352/7352 [==============================] - 0s - loss: 0.4158 - acc: 0.8301 - val_loss: 0.6631 - val_acc: 0.7917
Epoch 3/10
7352/7352 [==============================] - 0s - loss: 0.3455 - acc: 0.8641 - val_loss: 0.6318 - val_acc: 0.8188
Epoch 4/10
7352/7352 [==============================] - 0s - loss: 0.2961 - acc: 0.8852 - val_loss: 0.6573 - val_acc: 0.8222
Epoch 5/10
7352/7352 [==============================] - 0s - loss: 0.2710 - acc: 0.8930 - val_loss: 0.6858 - val_acc: 0.8168
Epoch 6/10
7352/7352 [==============================] - 0s - loss: 0.2439 - acc: 0.9100 - val_loss: 0.6604 - val_acc: 0.8286
Epoch 7/10
7352/7352 [==============================] - 0s - loss: 0.2389 - acc: 0.9060 - val_loss: 0.8035 - val_acc: 0.8018
Epoch 8/10
7352/7352 [==============================] - 0s - loss: 0.2223 - a

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,510,0,0,0,0,27
SITTING,0,452,14,1,0,24
STANDING,0,273,250,2,0,7
WALKING,0,5,0,406,68,17
WALKING_DOWNSTAIRS,0,9,0,42,355,14
WALKING_UPSTAIRS,0,0,0,46,17,408


In [13]:
model_base_mlp.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128, 32)           320       
_________________________________________________________________
reshape_1 (Reshape)          (None, 4096)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 24582     
Total params: 24,902
Trainable params: 24,902
Non-trainable params: 0
_________________________________________________________________


### 4.2 *Long Short Time Memory (LSTM)*
Test d'un réseau avec couche LSTM avec l'idée d'appréhender la structure temporelle des données.

#### *Statelesss Mode*

In [44]:
n_hidden = 32
#default stateful = False

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 6

batch_size=64
#else:
model_base_lstm =km.Sequential()
model_base_lstm.add(kl.LSTM(n_hidden, input_shape=(timesteps, input_dim), stateful=False))
model_base_lstm.add(kl.Dense(n_classes, activation='softmax'))

model_base_lstm.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model_base_lstm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 32)                5376      
_________________________________________________________________
dense_20 (Dense)             (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [45]:
# Default shuffle = True Meilleur avec Shuffle m True
t_start = time.time()
model_base_lstm.fit(X_train,  Y_train_dummies, batch_size=batch_size, validation_data=(X_test, Y_test_dummies), epochs=epochs, shuffle=False)
t_end = time.time()
t_learning = t_end-t_start

score = model_base_lstm.evaluate(X_test, Y_test_dummies)[1] 
print("Score With Simple MLP on Multidimensional Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
metadata_lstm = {"time_learning" : t_learning, "score" : score}
base_lstm_prediction = model_base_lstm.predict(X_test)

my_confusion_matrix(Y_test_dummies, base_lstm_prediction)

Train on 7352 samples, validate on 2947 samples
Epoch 1/10
7352/7352 [==============================] - 22s - loss: 1.4789 - acc: 0.3935 - val_loss: 1.2950 - val_acc: 0.4880
Epoch 2/10
7352/7352 [==============================] - 20s - loss: 1.1747 - acc: 0.5057 - val_loss: 1.2254 - val_acc: 0.5120
Epoch 3/10
7352/7352 [==============================] - 21s - loss: 1.1418 - acc: 0.5031 - val_loss: 1.1692 - val_acc: 0.5389
Epoch 4/10
7352/7352 [==============================] - 21s - loss: 1.0597 - acc: 0.5605 - val_loss: 1.0674 - val_acc: 0.5467
Epoch 5/10
7352/7352 [==============================] - 21s - loss: 0.9046 - acc: 0.6457 - val_loss: 0.9655 - val_acc: 0.5959
Epoch 6/10
7352/7352 [==============================] - 21s - loss: 0.8009 - acc: 0.6613 - val_loss: 0.9069 - val_acc: 0.6312
Epoch 7/10
7352/7352 [==============================] - 20s - loss: 0.7589 - acc: 0.6742 - val_loss: 0.8364 - val_acc: 0.6780
Epoch 8/10
7352/7352 [==============================] - 20s - loss: 0.

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,510,0,0,0,0,27
SITTING,0,345,118,23,3,2
STANDING,0,45,417,60,3,7
WALKING,0,0,1,297,45,153
WALKING_DOWNSTAIRS,0,0,0,50,208,162
WALKING_UPSTAIRS,0,0,1,49,77,344


#### *Statefull Mode*

In [15]:
model_base_lstm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [16]:
model_path = DATA_DIR+"keras_model/LSTM_dense.h5"
if os.path.isfile(model_path):
    model_base_lstm_92 = km.load_model(model_path)
    print(model_base_lstm_92.evaluate(X_test,Y_test_dummies))
    print(model_base_lstm_92.summary())

2944/2947 [============================>.] - ETA: 0s[0.49561213459482323, 0.92093654563963356]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________
None


### 4.3 Réseau avec couche convolutionelle (*ConvNet*)

In [1]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 6

X_train_conv = X_train.reshape(N_train, timesteps, input_dim, 1)
X_test_conv = X_test.reshape(N_test, timesteps, input_dim, 1)

#else:
model_base_conv_2D =km.Sequential()
model_base_conv_2D.add(kl.Conv2D(32, (3, 9), activation='relu', input_shape=(timesteps, input_dim, 1)))
model_base_conv_2D.add(kl.MaxPooling2D(pool_size=(2, 1)))
model_base_conv_2D.add(kl.Flatten())
model_base_conv_2D.add(kl.Dense(n_classes, activation='softmax'))
model_base_conv_2D.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

t_start = time.time()
model_base_conv_2D.fit(X_train_conv,  Y_train_dummies, batch_size=batch_size, validation_data=(X_test_conv, Y_test_dummies), epochs=epochs)
t_end = time.time()
t_learning = t_end-t_start

score = model_base_conv_2D.evaluate(X_test_conv, Y_test_dummies)[1] 
print("Score With Conv on Multidimensional Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
metadata_conv = {"time_learning" : t_learning, "score" : score}
base_conv_2D_prediction = model_base_conv_2D.predict(X_test_conv)

my_confusion_matrix(Y_test_dummies, base_conv_2D_prediction)

NameError: name 'X_train' is not defined

In [20]:
model_base_conv_2D.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 1, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 1, 32)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 2016)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 12102     
Total params: 12,998
Trainable params: 12,998
Non-trainable params: 0
_________________________________________________________________


**Objectif** trouver la meilleure architecture.

**Attention au sur-apprentissage** A force de rechercher la meilleure architecture en minimisant l'erreur sur l'échantillon test, celle finalement trouvée peut y être très adaptée réduisant ainsi la capacité de généralisation. Il serait prudent de multiplier le découpage de l'échantillon par validation croisée *Monte Carlo*.